In [29]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [30]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 3000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [31]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [32]:
from datasets import load_dataset

In [33]:
# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("venkycs/crime-story-dataset", split="train")

In [35]:
def format_instruction(sample):
    return f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

    {sample['input']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

    {sample['output']}<|eot_id|>"""

In [36]:
'''def generate_and_tokenize_prompt(data_point):
    prompt = format_instruction(data_point)
    return tokenizer(prompt, truncation=True, max_length=max_seq_length, padding="max_length")

dataset = dataset.shuffle().map(generate_and_tokenize_prompt)'''

'def generate_and_tokenize_prompt(data_point):\n    prompt = format_instruction(data_point)\n    return tokenizer(prompt, truncation=True, max_length=max_seq_length, padding="max_length")\n\ndataset = dataset.shuffle().map(generate_and_tokenize_prompt)'

In [37]:
def generate_and_tokenize_prompt(data_point):
    # Check for empty input or output and skip this datapoint if either is empty.
    if not data_point['input'] or not data_point['output']:
        return None  # Return None to filter out this sample
    prompt = format_instruction(data_point)
    return tokenizer(prompt, truncation=True, max_length=max_seq_length, padding="max_length")

# Filter out None values from the mapped dataset
dataset = dataset.shuffle().map(generate_and_tokenize_prompt, remove_columns=["input", "output"],num_proc=4).filter(lambda example: example is not None)


Map (num_proc=4):   0%|          | 0/26395 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26395 [00:00<?, ? examples/s]

In [34]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq

In [38]:
from unsloth import is_bfloat16_supported

In [39]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,  # Adjust for your GPU
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,  # Or num_train_epochs=1 for full training
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
)

In [40]:
# Make sure this cell is executed before the cell using SFTTrainer
from trl import SFTTrainer

In [41]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True, return_tensors="pt"),
    max_seq_length=max_seq_length,
    args=training_args,
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/26395 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/26395 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/26395 [00:00<?, ? examples/s]

In [42]:
from unsloth.chat_templates import train_on_responses_only

In [43]:
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/26395 [00:00<?, ? examples/s]

In [44]:
trainer_stats = trainer.train()
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 26,395 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,5.457800
2,6.136600
3,4.204800
4,4.732200
5,4.376600
6,4.350900
7,3.663700
8,3.650000
9,4.629800
10,3.786800


TrainOutput(global_step=60, training_loss=3.279210821787516, metrics={'train_runtime': 2033.9259, 'train_samples_per_second': 0.236, 'train_steps_per_second': 0.029, 'total_flos': 2.456404918272e+16, 'train_loss': 3.279210821787516})


In [45]:
# Inference
from unsloth.chat_templates import get_chat_template # Import the function

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",  #MUST BE llama-3.1 for Unsloth templates to be correct
)
FastLanguageModel.for_inference(model)  # Native 2x faster inference

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [46]:
def generate(prompt):
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to("cuda")
    outputs = model.generate(
        input_ids=inputs, max_new_tokens=256, use_cache=True, temperature=1.5, min_p=0.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [47]:
# Example Usage
test_prompt = "Summarize the following text: The quick brown rabbit jumps over the lazy frogs with no effort."
print(f"Prompt: {test_prompt}\n")
print(f"Response: {generate(test_prompt)}")



Prompt: Summarize the following text: The quick brown rabbit jumps over the lazy frogs with no effort.

Response: system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Summarize the following text: The quick brown rabbit jumps over the lazy frogs with no effort.assistant

This text appears to be a sentence parodying the classic phrase "The quick brown fox jumps over the lazy dog." The phrase you provided substitutes the word "frog" for "dog" to create a humorous and whimsical version.


In [ ]:

# 7. Saving the model
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

In [48]:
test_prompt = "Title: **The Fall of Arjun Singh: A Journey Through Shadows**\n\n---\n\nI still remember the first time I laid eyes on the vibrant streets of New Delhi. The aroma of freshly cooked parathas wafting through the air, the sound of honking cars mingling with rickshaw bells, and the blend of colors from street vendors selling everything from bangles to books—it was intoxicating. This was my playground, the backdrop against which I was raised. My name is Arjun Singh, though most people know me as A.S. Little did I know back then that this vivid city, filled with dreams and aspirations, would also be the stage for my fall from grace.\n\n#### A Middle-Class Beginnings\n\nBorn into a middle-class family in Dharmanpur, my upbringing was simple yet fulfilling. My father worked as a government clerk, and my mother was a teacher. We lived in a modest apartment, but I recall the warmth of my parents’ laughter and the evenings spent sharing stories of triumph and aspirations. Education was the cornerstone of our family values. I studied diligently at St. Mark's School, where academic achievements were highly coveted. The thrill of receiving accolades and prizes fueled my ambition. It was in those classrooms that I first learned about negotiation and the sweet taste of success. \n\nAfter completing my schooling, I pursued a Bachelor’s degree in Business Management at Delhi University. Those years were transformative. I joined various societies, honing my skills in public speaking and negotiation, often charming professors and peers alike. I felt invincible, cocky even. Graduation in 2007 should have been a milestone—a leap toward a shared dream with my fellow graduates of becoming an impactful player in the business world.\n\n#### The Bright Allure of Real Estate\n\nI ventured into real estate, what I believed would be a golden ticket. The market was booming, and for a few years, it felt as if I was living a dream. With natural charisma and a knack for closing deals, I quickly made a name for myself. People said I had the Midas touch; everything I approached seemed to turn to gold. I sold my first property within months and began climbing the elusive ladder of success. I networked tirelessly with affluent businessmen and politicians, woven into a world of opportunities. The property market flourished, and I basked in the euphoria of my achievements.\n\nBut as they say, fortune is fickle. The whispers of decline began to echo in 2015 when the Indian real estate market began to take a downturn. Mired in financial pressure, I faced dwindling profits and accumulating debts. Panic seeped into my professional life, and the desperation clawed at my ambition, turning my bright future into a shadow of what it could have been.\n\n#### The Descent into Darkness\n\nIt was during this tumultuous time I crossed paths with Vikrant Rao. The first time I met him, I was captivated by his presence—a man who exuded power and authority. He was notorious, a figure shrouded in mystery, whispered about in industry circles. I was teetering on the edge of financial ruin when Vikrant offered me an escape route. He extended a helping hand laced with temptation that I should have refused but couldn’t. \n\nThe plunge into the underworld began slowly, like a gentle descent into a pool of darkness. Vikrant introduced me to ‘The Black Syndicate,’ an organization that promised financial relief in exchange for loyalty and participation in their illicit schemes. I was a reluctant recruit at first, but in the face of financial devastation, the allure of quick cash seduced me. I began with minor offenses—fraudulent land purchases. Each little gamble paid off, and with it, my appetite for risk grew.\n\nEventually, my involvement escalated, morphing into drug trafficking and arms smuggling. I forged connections with individuals who operated on the fringes of legality, becoming adept at navigating through murky waters. The innocence of that first deal faded away, replaced with a dark, compulsive greed that overshadowed my moral compass.\n\n#### Building a Criminal Empire\n\nMy operations expanded exponentially as I became entrenched in the drug trade. I honed my skills in negotiations, adeptly working with corrupt politicians and affluent businessmen, all too eager to turn a blind eye for the right price. Cocaine and heroin became my commodities of choice. I meticulously established a supply chain that extended from Southeast Asia, building ties with drug manufacturers and arms dealers. \n\nTraveling to countries like Thailand and Malaysia under the guise of business meetings became routine. My real purpose? Establish connections with suppliers who could sustain my growing empire. Each trip turned into an intricate ballet of meetings, some in lavish hotels where deals were sealed with a handshake, others in dark alleyways where shadows whispered of danger. But I wore my bravado like armor and convinced myself I was merely playing the game.\n\nAs my empire grew, so did my network. I discovered a knack for recruitment, luring jobless young men desperate for financial relief. The promise of money and the status of being part of something larger than themselves drew them in like moths to a flame. But with each life pulled into my underworld, I felt the weight of moral decay.\n\nAt home, my family continued to live blissfully unaware of the dark undertones of my life. I maintained a front that masked my operations, often dismissing their concerns about my late nights and unexplained absences. The pride I felt in providing for them overshadowed the reality of my actions. My permanent address in Dharmanpur became a fortress that shielded my illicit activities from prying eyes. I was cautious, calculating even, managing a double life that allowed me to tread between two worlds.\n\n#### The Tipping Point\n\nHowever, as the drug problem in Noida grew increasingly dire, the law enforcement agencies began tightening their grip. Whispers of increased surveillance turned into a palpable threat, casting shadows of unease over my operations. I found myself walking a tightrope, aware that the fall could be catastrophic.\n\nOn September 15, 2023, that tightrope snapped. I had arranged a major transaction—the sale of 1.5 kg of heroin in Sector 32, Noida, a deal that promised to be one of the most lucrative yet. Confident in my plan and in the loyalty of my associates, I felt invincible until the moment I was confronted by law enforcement officials. Officers Rajiv Kumar and his team swarmed in, and the façade of my empire came crashing down.\n\nThe arrest was swift—1.5 kg of heroin, three firearms, and substantial cash were seized. My mind raced through the implications of the moment. I was no longer a successful businessman caught in a downturn; I was a criminal deeply entrenched in the darkest corners of society. The whispers that had become my allies now turned into wails of betrayal and accountability. \n\n#### Chaos and Revelations\n\nFollowing my arrest, the investigation painted a harsh picture of my operations. Authorities dug into every crevice of my life, uncovering the network of associates I had woven throughout the underworld: Ravi Kapoor and Sameer Pratap, men I had initially considered loyal allies. Their testimonies cast a wide net of betrayal; they snitched and spun tales of our dealings in a desperate bid for leniency. \n\nEach revelation felt like a dagger to my chest. I had meticulously curated a life of deceit, and now the world knew. News outlets buzzed with stories of Arjun Singh and ‘The Black Syndicate.’ I became a symbol of the corruption and moral bankruptcy that plagued the society I once thrived in. \n\nWhat haunted me more than the betrayal was the realization that I had abandoned the principles I had grown up on. My ambition had spiraled into an insatiable greed, one that blinded me to the consequences of my actions. Each dollar gained through those deceitful transactions was like a drop of poison, slowly eroding the very core of who I once was. \n\n#### Reflection in Captivity\n\nAs I sat in the cold confines of my prison cell, the weight of my decisions pressed heavily upon my conscience. I had once roamed the streets, a man who believed he controlled his fate, but now, I was a mere shadow of that persona. My life story was no longer a narrative of ascension but a cautionary tale of how one small choice could lead to a cascade of consequences.\n\nI thought about my family, my parents, who had instilled values in me—values I had so callously discarded for fleeting gains. Would they recognize the man I had become? Shame welled within me, and I wondered if the innocence of my upbringing could ever reconcile with the darkness I had embraced. Guilt consumed me day in, day out, as I replayed the moments that led me to this hellish reality in my mind.\n\n#### Facing the Future\n\nMy journey through shadows is far from over. The investigations still unfurl, and the authorities are piecing together the labyrinth of my dealings. I realized that what began as a chase for success had morphed into an entrapment of my making. \n\nThere were whispers of possible charges that would envelop me, potentially for years. In those quiet moments of reflection, I contemplated what awaited me—a future I had traded for a brief moment of power. \n\nMore than the prisons and the consequences I now face, it struck me that true redemption lies in confronting the darkness and acknowledging the part I played in a vast societal issue—a systemic decay I had fueled as much as I had benefitted from. \n\nCould I ever truly redeem myself in the eyes of those I had wronged? Could there be any semblance of hope, or was I too far gone, a figure forever encircled by the shadows of my past? The future has become an enigma, shrouded in uncertainty and marked by the choices that had led me here. I could only wait and face whatever consequences lay ahead, hoping for a future that would one day make sense of the chaos I had created.\n\nThe vibrant streets of New Delhi that once represented possibility and promise felt like a lifetime away. My name, once celebrated, was now a symbol of caution—a stark reminder of how ambition unchecked could lead one to embrace shadows."

In [49]:
print(f"Prompt: {test_prompt}\n")


Prompt: Title: **The Fall of Arjun Singh: A Journey Through Shadows**

---

I still remember the first time I laid eyes on the vibrant streets of New Delhi. The aroma of freshly cooked parathas wafting through the air, the sound of honking cars mingling with rickshaw bells, and the blend of colors from street vendors selling everything from bangles to books—it was intoxicating. This was my playground, the backdrop against which I was raised. My name is Arjun Singh, though most people know me as A.S. Little did I know back then that this vivid city, filled with dreams and aspirations, would also be the stage for my fall from grace.

#### A Middle-Class Beginnings

Born into a middle-class family in Dharmanpur, my upbringing was simple yet fulfilling. My father worked as a government clerk, and my mother was a teacher. We lived in a modest apartment, but I recall the warmth of my parents’ laughter and the evenings spent sharing stories of triumph and aspirations. Education was the corner

In [50]:
print(f"Response: {generate(test_prompt)}")

Response: system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Title: **The Fall of Arjun Singh: A Journey Through Shadows**

---

I still remember the first time I laid eyes on the vibrant streets of New Delhi. The aroma of freshly cooked parathas wafting through the air, the sound of honking cars mingling with rickshaw bells, and the blend of colors from street vendors selling everything from bangles to books—it was intoxicating. This was my playground, the backdrop against which I was raised. My name is Arjun Singh, though most people know me as A.S. Little did I know back then that this vivid city, filled with dreams and aspirations, would also be the stage for my fall from grace.

#### A Middle-Class Beginnings

Born into a middle-class family in Dharmanpur, my upbringing was simple yet fulfilling. My father worked as a government clerk, and my mother was a teacher. We lived in a modest apartment, but I recall the warmth of my parents’ laughter and the eve

In [51]:
repo_id ="Subhi09/unsloth_finetuned"

In [52]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [54]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `new` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when

In [58]:
model.push_to_hub(repo_id, commit_message="Add LoRA model",use_auth_token=True)
tokenizer.push_to_hub(repo_id, commit_message="Add tokenizer",use_auth_token=True)

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Subhi09/unsloth_finetuned


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]